In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import Quandl
import functools
import seaborn as sns
import ggplot as gg
import statsmodels.api as sm

# set parameters
M =20
j_optimal = [0.0017,0.0017,0.0019, 0.0029,  0.0001,0.0011, 0.0029,0.0019, 0.0019,0.0027]
g_optimal = [0.0067,0.0035, 0.0069,0.0057,0.0003,0.0081,0.0099 ,0.0035,0.0023,0.0069]
s = float('inf') # s is pretty much useless here

ETF_pair = [['EOD/RYU', 'EOD/XLU'],['EOD/IST', 'EOD/IYZ'],['EOD/RING', 'EOD/GDX'],['EOD/XSD', 'EOD/SMH'],['EOD/PBE', 'EOD/XBI'],
           ['EOD/IEO', 'EOD/XOP'],['EOD/PXJ', 'EOD/OIH'],['EOD/RTH', 'EOD/XRT'],['EOD/SIVR', 'EOD/SLV'],['EOD/HYLD', 'EOD/JNK']]

Name_1 = ['RYU','IST','RING','XSD','PBE','IEO','PXJ','RTH','SIVR','HYLD']
Name_2 = ['XLU','IYZ','GDX','SMH','XBI','XOP','OIH','XRT','SLV','JNK']

Label = ['0. RYU XLU','1. IST IYZ','2. RING GDX','3. XSD SMH','4. PBE XBI','5. IEO XOP','6. PXJ OIH','7. RTH XRT','8. SIVR SLV',
'9. HYLD JNK']



ModuleNotFoundError: No module named 'Quandl'

In [ ]:
for i in range(0,10):
    raw_data = Quandl.get(ETF_pair[i], trim_start='2013-12-02', trim_end='2015-12-31',authtoken='vVPRRRiSWG3RxCPMupfa')
    def clean_quandl_columns(dataframe):
        replacement_columns = {}
        for c in dataframe.columns:
            series_name, variable = c.split(' - ')
            source_name, asset = series_name.split('.')
            replacement_columns[c] = asset + "_" + variable
            renamed_data = dataframe.rename(columns=replacement_columns)
        return renamed_data
    renamed_data = clean_quandl_columns(raw_data)

    data = pd.DataFrame({Name_1[i] + '_AJClose':renamed_data[Name_1[i] + '_Adj_Close'],Name_2[i] + '_AJClose':renamed_data[Name_2[i] + '_Adj_Close'],
         Name_1[i] + '_VOLUME':renamed_data[Name_1[i] + '_Volume'],Name_2[i] + '_VOLUME':renamed_data[Name_2[i] + '_Volume']})
    data['NT'] = float('nan')
    
    for index in range(15,len(data)):
        data['NT'][index] = (data[Name_1[i] + '_AJClose'] * data[Name_1[i] + '_VOLUME'])[index - 15:index - 1].median()
    Result = []
    Up = (data[Name_1[i] + '_AJClose'][M:].values - data[Name_1[i] + '_AJClose'][0: - M].values)/data[Name_1[i] + '_AJClose'][0: - M].values
    Down = (data[Name_2[i] + '_AJClose'][M:].values - data[Name_2[i] + '_AJClose'][0: - M].values)/data[Name_2[i] + '_AJClose'][0: - M].values
    Return = Up - Down
    
    # trade PnL
    PnL = np.empty((len(data)))
    PnL.fill(np.nan)
    PnL[M + 1:] = Return[: - 1]
    
    data['Profit'] = PnL
    data['Buy_signal'] = data.Profit>g_optimal[i];
    data['Sell_signal'] = data.Profit<j_optimal[i];
    
    # determine last day of the month
    from datetime import date
    Last_Day = pd.date_range('1/1/2014', periods=24, freq='BM')
    for date in Last_Day:
        data.loc[date,'Sell_signal'] = True
    data['X']= 0
    data['Y']= 0
    data[Name_1[i] + '_Return'] = (data[Name_1[i] + '_AJClose']  -  data.shift(1)[Name_1[i] + '_AJClose']) 
    data[Name_2[i] + '_Return'] = (data[Name_2[i] + '_AJClose']  -  data.shift(1)[Name_2[i] + '_AJClose']) 
    s =  - float('Inf')
    data['Daily_Gain'] = 0
    
    # trade strategy
    for index in data.index:  
        if data.loc[index,'Buy_signal'] and data.shift(1).loc[index,'X'] == 0:
            data.loc[index,'X'] =  - int(round(data.loc[index,'NT']/100/data.loc[index,Name_1[i] + '_AJClose']))
            data.loc[index,'Y'] = int(round(data.loc[index,'NT']/100/data.loc[index,Name_2[i] + '_AJClose']))
        else:
            data.loc[index,'X'] = data.shift(1).loc[index,'X']
            data.loc[index,'Y'] = data.shift(1).loc[index,'Y']
        daily_gain = data.loc[index,'X'] * data.loc[index,Name_1[i] + '_Return'] + data.loc[index,'Y'] * data.loc[index,Name_2[i] + '_Return']
        data.loc[index,'Daily_Gain'] = daily_gain
        if data.loc[index,'Sell_signal'] or daily_gain < s: # stop loss
            data.loc[index,'X'] = 0
            data.loc[index,'Y'] = 0
            data.loc[index,'Sell_signal'] = True
    K = 2 * np.nanmax(data['NT'])
    if 'ReturnMatrix' in locals() or 'ReturnMatrix' in globals():
        ReturnMatrix[Label[i]] = data['Daily_Gain']/K
    else:
        ReturnMatrix = pd.DataFrame(index = data.index)
        ReturnMatrix[Label[i]] = data.Daily_Gain/K


In [ ]:
# get KF data
KFData = Quandl.get('KFRENCH/FACTORS_D',authtoken='vrJkt5c3B5qJRExuYYPg', trim_start='2013 - 12 - 02',
                  trim_end='2015 - 12 - 31',)
KFData.head()

In [2]:
# regression

ReturnMatrix = ReturnMatrix.dropna()
ReturnMatrix.shape

residual_Mkt = pd.DataFrame(index = ReturnMatrix.index)
residual_SMB = pd.DataFrame(index = ReturnMatrix.index)
residual_HML = pd.DataFrame(index = ReturnMatrix.index)
residual_RF = pd.DataFrame(index = ReturnMatrix.index)
residual_Hedged = pd.DataFrame(index = ReturnMatrix.index)

for n in range(0,10):
    residual_Mkt[Label[n]] = sm.OLS(ReturnMatrix[Label[j]],sm.add_constant(KFData.loc[ReturnMatrix.index]['Mkt-RF'])).fit().resid
    residual_SMB[Label[n]] = sm.OLS(ReturnMatrix[Label[j]],sm.add_constant(KFData.loc[ReturnMatrix.index]['SMB'])).fit().resid
    residual_HML[Label[n]] = sm.OLS(ReturnMatrix[Label[j]],sm.add_constant(KFData.loc[ReturnMatrix.index]['HML'])).fit().resid
    residual_RF[Label[n]] = sm.OLS(ReturnMatrix[Label[j]],sm.add_constant(KFData.loc[ReturnMatrix.index]['RF'])).fit().resid
    residual_Hedged[Label[n]] = sm.OLS(ReturnMatrix[Label[j]],sm.add_constant(KFData.loc[ReturnMatrix.index])).fit().resid



NameError: name 'ReturnMatrix' is not defined

In [3]:
# sharpe & sortino ratios

Sharpe = np.mean(ReturnMatrix)/np.std(ReturnMatrix)
Sortino = (np.mean(ReturnMatrix) - np.mean(residual_RF))/np.std(ReturnMatrix)

Sharpe_Mkt = np.mean(ReturnMatrix)/np.std(residual_Mkt)
Sortino_Mkt = (np.mean(ReturnMatrix) - np.mean(residual_RF))/np.std(residual_Mkt)

Sharpe_SMB = np.mean(ReturnMatrix)/np.std(residual_SMB)
Sortino_SMB = (np.mean(ReturnMatrix) - np.mean(residual_RF))/np.std(residual_SMB)

Sharpe_HML = np.mean(ReturnMatrix)/np.std(residual_HML)
Sortino_HML = (np.mean(ReturnMatrix)- np.mean(residual_RF))/np.std(residual_HML)

Sharpe_Hedged = np.mean(ReturnMatrix)/np.std(residual_Hedged)
Sortino_Hedged = (np.mean(ReturnMatrix) - np.mean(residual_RF))/np.std(residual_Hedged)


result = pd.concat([Sharpe, Sharpe_Mkt,Sharpe_SMB,Sharpe_HML, Sharpe_Hedged], axis=1, join_axes=[Sharpe.index])

result2 = pd.concat([Sortino, Sortino_Mkt,Sortino_SMB,Sortino_HML,Sortino_Hedged], axis=1, join_axes=[Sharpe.index])

my_columns = ["Sharpe_Orig",  "Sharpe_Mkt","Sharpe_SMB","Sharpe_HML","Sharpe_Hedged"]
my_columns2 = ["Sortino_Orig","Sortino_Mkt","Sortino_SMB","Sortino_HML", "Sortino_Hedged"]

result.columns = my_columns
result


NameError: name 'ReturnMatrix' is not defined

Using Sharpe ratio as a measure of performance:

We can see from the chart above, for group 1, 2, 4, 6, 8 and 9, there is definite improvement in any kind of hedging, and at most time, heding with all Fama-French factors simultaneously will lead to the most improvement. However, for group 0, 3, 5 and 7, after heding the performance become worse in any way of hedging. 

In [82]:
result2.columns = my_columns2
result2

,Sortino_Orig,Sortino_Mkt,Sortino_SMB,Sortino_HML,Sortino_Hedged
0. RYU XLU,-0.062125,-0.117870,-0.118176,-0.117878,-0.118187
1. IST IYZ,0.030126,0.248396,0.249039,0.248411,0.249063
2. RING GDX,0.046203,0.263962,0.264646,0.263978,0.264672
3. XSD SMH,-0.030385,-0.271327,-0.272029,-0.271343,-0.272056
4. PBE XBI,0.032236,0.336212,0.337083,0.336233,0.337116
5. IEO XOP,-0.038759,-0.426903,-0.428008,-0.426928,-0.428050
6. PXJ OIH,0.094747,0.186800,0.187284,0.186812,0.187302
7. RTH XRT,-0.089681,-0.667984,-0.669713,-0.668024,-0.669780
8. SIVR SLV,0.097895,0.112551,0.112842,0.112557,0.112853
9. HYLD JNK,0.000000,0.000000,0.000000,0.000000,0.000000


Using Sortino ratio as a measure of performance:

similar results as using sharpe ratio